In [1]:
import jieba
import pandas as pd
import gensim
import logging
import re
import csv
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%load_ext autoreload
%autoreload 2

In [57]:
textbook_path = '../tw_clean.txt'
#df = pd.read_csv(textbook_path, sep='\n', names=['documents'], header=None)

In [58]:
def get_corpus(fname):
    with open(fname) as f:
        lines = iter(f)
        #lines.next()
        for line in lines:
            yield line

class CorpusFriendly(object):
    def __init__(self,fname, tag=True, topK=None):
        self.topK = topK
        self.fname = fname
        self.tag = tag
    def __iter__(self):
        k = 1
        corpus_memory_friendly = get_corpus(self.fname)
        for context in corpus_memory_friendly:
            #data = gensim.utils.to_unicode(data).split(',')
            seg_context = jieba.cut(context, cut_all=True)
            words = " ".join(seg_context).split()
            #print ", ".join(words)
            label = [str(k)]
            if self.tag:
                yield gensim.models.doc2vec.TaggedDocument(words, label)
            else:
                yield words
            if self.topK:
                if k >= self.topK:
                    break
                k += 1

In [59]:
# corpus_train = CorpusFriendly(fname='hk_clean.txt',topK=3)
# for c in corpus_train:
#     print ", ".join(c.words)

In [60]:
# for c in get_corpus(textbook_path):
#     print c

## Create Corpus for Word2vec

In [61]:
text = []
for c in get_corpus(textbook_path):
    text.extend(c)

In [62]:
text_sentences = []
for t in "".join(text).split('\n'):
    text_sentences.extend(t.split())

In [63]:
corpus_word2vec = []
for t in text_sentences:
    corpus_word2vec.extend(t.split('。'))
corpus_word2vec = filter(None, corpus_word2vec)

In [64]:
print len(corpus_word2vec)

2570


In [65]:
with open("../tw_clean_sentences.txt", 'wb') as outfile:
    writer = csv.writer(outfile, delimiter='\n')
    for t in corpus_word2vec:
        writer.writerow([t])

## Word2vec

In [66]:
num_features = 2000
min_word_count = 1
num_workers = 4
context = 10

In [68]:
corpus_train = CorpusFriendly(fname="../tw_clean_sentences.txt", tag=False)
model_word2vec = gensim.models.word2vec.Word2Vec(corpus_train, 
                                        size=num_features, 
                                        min_count=min_word_count, 
                                        iter=20, 
                                        window=context, 
                                        workers=4)

2016-12-29 14:13:31,599 : INFO : collecting all words and their counts
2016-12-29 14:13:31,601 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-12-29 14:13:31,856 : INFO : collected 12223 word types from a corpus of 70494 raw words and 2570 sentences
2016-12-29 14:13:31,884 : INFO : min_count=1 retains 12223 unique words (drops 0)
2016-12-29 14:13:31,884 : INFO : min_count leaves 70494 word corpus (100% of original 70494)
2016-12-29 14:13:31,917 : INFO : deleting the raw counts dictionary of 12223 items
2016-12-29 14:13:31,918 : INFO : sample=0.001 downsamples 22 most-common words
2016-12-29 14:13:31,918 : INFO : downsampling leaves estimated 65230 word corpus (92.5% of prior 70494)
2016-12-29 14:13:31,919 : INFO : estimated required memory for 12223 words and 2000 dimensions: 201679500 bytes
2016-12-29 14:13:31,946 : INFO : resetting layer weights
2016-12-29 14:13:32,435 : INFO : training model with 4 workers on 12223 vocabulary and 2000 features, using 

In [69]:
result_word2vec = model_word2vec.most_similar(positive=[u'毛泽东'], topn=10)
for e in result_word2vec:
    print e[0], e[1]

2016-12-29 14:13:44,829 : INFO : precomputing L2-norms of word weight vectors


派对 0.997701048851
般 0.997641205788
时局 0.997374773026
少年 0.997246026993
这 0.996866822243
卫兵 0.996851146221
群众路线 0.996850311756
通常 0.996457397938
围观 0.996254146099
红卫兵 0.99623644352


## Doc2Vec

In [70]:
corpus_train = CorpusFriendly(fname=textbook_path)
model_doc2vec = gensim.models.doc2vec.Doc2Vec(size=2000, min_count=1, iter=30, window=10, workers=6)
model_doc2vec.build_vocab(corpus_train)

2016-12-29 14:13:51,869 : INFO : collecting all words and their counts
2016-12-29 14:13:51,953 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2016-12-29 14:13:52,111 : INFO : collected 12223 word types and 1 unique tags from a corpus of 267 examples and 70494 words
2016-12-29 14:13:52,185 : INFO : min_count=1 retains 12223 unique words (drops 0)
2016-12-29 14:13:52,186 : INFO : min_count leaves 70494 word corpus (100% of original 70494)
2016-12-29 14:13:52,215 : INFO : deleting the raw counts dictionary of 12223 items
2016-12-29 14:13:52,216 : INFO : sample=0 downsamples 0 most-common words
2016-12-29 14:13:52,216 : INFO : downsampling leaves estimated 70494 word corpus (100.0% of prior 70494)
2016-12-29 14:13:52,217 : INFO : estimated required memory for 12223 words and 2000 dimensions: 204132300 bytes
2016-12-29 14:13:52,232 : INFO : constructing a huffman tree from 12223 words
2016-12-29 14:13:52,564 : INFO : built huffman tree with maximum node dept

In [71]:
model_doc2vec.train(corpus_train)

2016-12-29 14:13:53,748 : INFO : training model with 6 workers on 12223 vocabulary and 2000 features, using sg=0 hs=1 sample=0 negative=0
2016-12-29 14:13:53,748 : INFO : expecting 267 sentences, matching count from corpus used for vocabulary survey
2016-12-29 14:13:54,934 : INFO : PROGRESS: at 4.11% examples, 79217 words/s, in_qsize 0, out_qsize 0
2016-12-29 14:13:55,950 : INFO : PROGRESS: at 10.80% examples, 99632 words/s, in_qsize 0, out_qsize 0
2016-12-29 14:13:56,965 : INFO : PROGRESS: at 20.04% examples, 106243 words/s, in_qsize 0, out_qsize 0
2016-12-29 14:13:58,007 : INFO : PROGRESS: at 24.13% examples, 100966 words/s, in_qsize 1, out_qsize 0
2016-12-29 14:13:59,176 : INFO : PROGRESS: at 31.95% examples, 100017 words/s, in_qsize 3, out_qsize 0
2016-12-29 14:14:00,222 : INFO : PROGRESS: at 38.63% examples, 101582 words/s, in_qsize 2, out_qsize 0
2016-12-29 14:14:01,287 : INFO : PROGRESS: at 45.29% examples, 101260 words/s, in_qsize 3, out_qsize 0
2016-12-29 14:14:02,352 : INFO :

1579800

In [45]:
#model_doc2vec.save('models/gensim_doc2vec_hk_clean')

In [44]:
result_doc2vec = model_doc2vec.most_similar(positive=[u'毛泽东'], topn=20)
for e in result_doc2vec:
    print e[0], e[1]

2016-12-29 14:10:25,190 : INFO : precomputing L2-norms of word weight vectors


大会 0.456900596619
中央 0.446250140667
召开 0.441353291273
邓小平 0.43362864852
七届 0.428785741329
文章 0.401267051697
救亡图存 0.397203177214
修正 0.390192836523
会议 0.385841727257
领导 0.374116033316
三个代表 0.373360186815
报告 0.370967835188
错误 0.369134247303
李大钊 0.36832010746
决定 0.365838527679
过渡时期 0.365819334984
两万多 0.360181629658
举行 0.354957491159
代表人 0.351537585258
思想 0.350723713636


## Find top N adjective from similar words 

In [18]:
import jieba.posseg

### MostSimilarGenerator
MostSimilarGenerator is a generator to efficiently find all the adj and adv

In [19]:
class MostSimilarGenerator(object):
    def __init__(self, model, word=u'毛泽东'):
        self.word= word
        self.model= model
    def __iter__(self):
        for sim_word in self.model.most_similar(self.word, topn=len(self.model.vocab)):
            yield sim_word

### is_adj()
is_adj() will return True if input is an adjective, an adverb or an idiom, otherwise False

jieba.posseg actually has ability to tag which part of speech is. I choose to tag adjective and adverb. Instead of using DictionaryServices, I find it makes more sense. However, we can always change the method.

In [53]:
def is_adj(word):
    word = jieba.posseg.cut(word)
    for w, f in word:
        if 'a' in f or 'd' in f or 'i' in f:
            return True
    #print w, f
    return False

### find_topn_adj()
find_topn_adj() will return a list of most similar words and corresponding scores of the word of interest 

In [54]:
def find_topn_adj(n, model, word=u'毛泽东',):
    k = 0
    topn_adj = []
    topn_score = []
    for sim_word in MostSimilarGenerator(model, word=word):
        if is_adj(sim_word[0]):
            topn_adj.append(sim_word[0])
            topn_score.append(sim_word[1])
            k += 1
        if k >= n:
            return zip(topn_adj, topn_score)

In [55]:
for w, s in find_topn_adj(n=20, model= model_word2vec, word=u'毛泽东'):
    print w, s

独秀 0.878687381744
令人鼓舞 0.859706223011
持久 0.83676224947
拨乱反正 0.793787062168
偏听偏信 0.792282879353
错误思想 0.781506538391
反正 0.779723525047
解放思想 0.775874376297
最低 0.77518415451
平明 0.744004607201
重新 0.73650676012
大民 0.729310035706
视死如归 0.719754815102
老 0.711652576923
幼稚 0.707704901695
实际上 0.702308535576
定将 0.700187206268
极 0.696991801262
平实 0.688151359558
深 0.678931713104


In [56]:
for w, s in find_topn_adj(n=20, model= model_doc2vec, word=u'毛泽东'):
    print w, s

救亡图存 0.397203177214
隆重 0.334757268429
大打出手 0.312203615904
正面 0.311421692371
大手 0.305309802294
一致 0.281828820705
各口岸 0.270859718323
明确 0.265751928091
同意 0.246563345194
远 0.233466386795
正确 0.232836142182
边 0.224828645587
在此之前 0.217532545328
之长 0.21311865747
民富 0.212604701519
反正 0.207456231117
最低 0.207350760698
严重错误 0.206992208958
不再 0.20612719655
星星之火 0.20379024744
